In [10]:
from htrc_features import Volume, FeatureReader

In [12]:
ids = ['uc1.31175037148668', "hvd.32044013656053"]

In [13]:
import htrc

In [14]:
Volume(id=ids[0])

ERROR:root:HTTP Error accessing http://data.analytics.hathitrust.org/features-2020.03/uc1\37346\uc1.31175037148668.json.bz2
Traceback (most recent call last):
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\caching.py", line 73, in open
    fout = super().open(id, **kwargs)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\resolvers.py", line 122, in open
    compression=compression,  **kwargs)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\resolvers.py", line 203, in _open
    return Path(dir, filename).open(mode = mode)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\pathlib.py", line 1208, in open
    opener=self._opener)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\pathlib.py", line 1063, in _opener
    return self._accessor.open(self, flags, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jaabi\\AppData\\Local\\Temp\\uc1.31175037148668.json.bz2'

During ha

HTTPError: HTTP Error 404: Not Found

In [8]:
htrc.volumes.download_volumes(volume_ids='uc1.31175037148668', output_dir='assets')

NoOptionError: No option 'capsule_id' in section: 'jwt'

In [15]:
import os
import requests

# Replace with the HTID you care about
htid = "uc1.31175037148668"

# Make output dirs
os.makedirs(f"images/{htid}", exist_ok=True)
os.makedirs(f"text/{htid}", exist_ok=True)

# Use the Feature Reader to get page count
from htrc_features import feature_reader, Volume
vol = Volume(htid)
page_count = vol.page_count

# Download loop
for seq in range(1, page_count + 1):
    # Image URL
    img_url = f"https://babel.hathitrust.org/cgi/imgsrv/image?id={htid}&seq={seq}&format=jpeg"
    r = requests.get(img_url)
    if r.status_code == 200:
        with open(f"images/{htid}/page_{seq:04d}.jpg", "wb") as f:
            f.write(r.content)

    # Text URL
    txt_url = f"https://babel.hathitrust.org/cgi/mb?a=gettext&seq={seq}&id={htid}"
    r = requests.get(txt_url)
    if r.status_code == 200:
        with open(f"text/{htid}/page_{seq:04d}.txt", "w", encoding="utf-8") as f:
            f.write(r.text)

    print(f"Downloaded page {seq}")

print("All done.")


ERROR:root:HTTP Error accessing http://data.analytics.hathitrust.org/features-2020.03/uc1\37346\uc1.31175037148668.json.bz2
Traceback (most recent call last):
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\caching.py", line 73, in open
    fout = super().open(id, **kwargs)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\resolvers.py", line 122, in open
    compression=compression,  **kwargs)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\site-packages\htrc_features\resolvers.py", line 203, in _open
    return Path(dir, filename).open(mode = mode)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\pathlib.py", line 1208, in open
    opener=self._opener)
  File "c:\Users\jaabi\anaconda3\envs\synthdog\lib\pathlib.py", line 1063, in _opener
    return self._accessor.open(self, flags, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jaabi\\AppData\\Local\\Temp\\uc1.31175037148668.json.bz2'

During ha

HTTPError: HTTP Error 404: Not Found

In [ ]:
import os
import requests

# Common browser headers
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    ),
    "Referer": "https://babel.hathitrust.org/",
    "Accept": "image/webp,image/apng,image/*,*/*;q=0.8"
}

def download_volume(htid, out_dir="downloads"):
    img_dir = os.path.join(out_dir, htid, "pages")
    txt_dir = os.path.join(out_dir, htid, "text")
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(txt_dir, exist_ok=True)

    page = 1
    while True:
        img_url = (
            f"https://babel.hathitrust.org/cgi/imgsrv/image?"
            f"id={htid}&seq={page}&format=jpeg"
        )
        img_resp = requests.get(img_url, headers=HEADERS)
        if img_resp.status_code != 200:
            break

        with open(f"{img_dir}/page_{page:04d}.jpg", "wb") as f:
            f.write(img_resp.content)

        txt_url = (
            f"https://babel.hathitrust.org/cgi/mb?"
            f"a=gettext&id={htid}&seq={page}"
        )

        print(txt_url)
        txt_resp = requests.get(txt_url, headers={"User-Agent": HEADERS["User-Agent"]})
        if txt_resp.status_code == 200:
            with open(f"{txt_dir}/page_{page:04d}.txt", "w", encoding="utf-8") as f:
                f.write(txt_resp.text)

        print(f"Downloaded page {page}")
        page += 1

    print(f"Finished downloading {page-1} pages for {htid}")

if __name__ == "__main__":
    target_htid = "uc1.31175037148668"
    download_volume(target_htid)


https://babel.hathitrust.org/cgi/mb?a=gettext&id=uc1.31175037148668&seq=1
Downloaded page 1
https://babel.hathitrust.org/cgi/mb?a=gettext&id=uc1.31175037148668&seq=2
Downloaded page 2
Finished downloading 2 pages for uc1.31175037148668


In [21]:
import os
import requests
from typing import List

GUTENDEX_API = "https://gutendex.com/books"

def get_book_ids_by_language(language_code: str, limit: int = 1) -> List[int]:
    """
    Query the Gutendex API to retrieve Gutenberg book IDs for the specified language.
    Returns up to `limit` IDs.
    """
    params = {"languages": language_code}
    ids = []
    page = 1

    while len(ids) < limit:
        params["page"] = page
        resp = requests.get(GUTENDEX_API, params=params)
        resp.raise_for_status()
        data = resp.json()

        for book in data["results"]:
            if len(ids) >= limit:
                break
            ids.append(book["id"])

        if not data["next"]:
            break
        page += 1

    return ids

def construct_gutenberg_urls(book_id: int) -> dict:
    """
    Given a Gutenberg book ID, construct the text and PDF URLs.
    """
    return {
        "text": f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt",
        "pdf":  f"https://www.gutenberg.org/files/{book_id}/{book_id}-pdf.pdf"
    }

def download_file(url: str, dest_path: str):
    """
    Download a file from `url` to `dest_path`. Creates parent directories if needed.
    """
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    resp = requests.get(url, stream=True)
    resp.raise_for_status()
    with open(dest_path, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Saved {dest_path}")

def download_books_by_language(language_code: str, limit: int = 1, out_dir: str = "gutenberg"):
    """
    Download up to `limit` books in the given language.
    Saves plain text and PDF under `out_dir/{language}/{book_id}/`.
    """
    ids = get_book_ids_by_language(language_code, limit)
    for book_id in ids:
        urls = construct_gutenberg_urls(book_id)
        book_dir = os.path.join(out_dir, language_code, str(book_id))
        
        # Download plain text
        text_dest = os.path.join(book_dir, f"{book_id}.txt")
        try:
            download_file(urls["text"], text_dest)
        except Exception as e:
            print(f"Failed to download text for {book_id}: {e}")

        # Download PDF
        pdf_dest = os.path.join(book_dir, f"{book_id}.pdf")
        try:
            download_file(urls["pdf"], pdf_dest)
        except Exception as e:
            print(f"Failed to download PDF for {book_id}: {e}")

if __name__ == "__main__":
    # Example: download 1 English and 1 Portuguese book
    download_books_by_language("en", limit=1)
    download_books_by_language("pt", limit=1)


Saved gutenberg\en\84\84.txt
Failed to download PDF for 84: 404 Client Error: Not Found for url: https://www.gutenberg.org/files/84/84-pdf.pdf
Saved gutenberg\pt\31552\31552.txt
Saved gutenberg\pt\31552\31552.pdf
